In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from DGP_Fed import Simulations_Fed, DGP_Fed, Plotter
import warnings
from collections import defaultdict
import pickle

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
sns.set_style("whitegrid")
sns.set_palette("Set2")
# Convert nested defaultdict to a regular dictionary
def convert_to_dict(d):
    if isinstance(d, defaultdict):
        d = {k: convert_to_dict(v) for k, v in d.items()}
    return d

def run_simulation(shared_parameters_dict, specific_parameters_rct, specific_parameters_obs, n_simus, n_blocks, meta_IVW=True, intercepts_shared_for_shot=True, add_H_cols=False, only_SGD=False, cross_device_scenario=False, show_locals=True, plot=True):
    # Regroup parameters
    if specific_parameters_rct is not None:
        clients_parameters_rct = {
            client_name: {
                **shared_parameters_dict,
                **specific_parameters_rct[client_name],
            }
            for client_name in specific_parameters_rct.keys()
        }
    if specific_parameters_obs is not None:
        clients_parameters_obs = {
            client_name: {
                **shared_parameters_dict,
                **specific_parameters_obs[client_name],
            }
            for client_name in specific_parameters_obs.keys()
        }
            

    params_of_simulation = {
        "n_simulations": n_simus,
        "fixed_design": False,
        "known_sigma2": True,
    }

    params_for_federation = {
        "max_rounds": 4000, # set it to 4000 for the paper, 10000 for Adjusted Gradient Descent
        "learning_rate": 0.01,
        "batch_size": None,
    }

    params_of_estimation = {
        "clip_or_trim": None,
        "estimator_family":["g_formula"],
        "estimators_to_compute": ["meta_SW", "meta_IVW", "1S_SW", '1S_IVW (true sigma2)', "SGD_fed", "diff_in_means"] if meta_IVW else ["meta_SW", "1S_SW", '1S_IVW (true sigma2)', "SGD_fed"],
        # "estimators_to_compute": ["diff_in_means"],
        "aggregations_to_perform": ["SW_agg"],
        "params_for_federation": params_for_federation,
        "ivw_with_hat_var": True,
        "intercepts_shared_for_shot": intercepts_shared_for_shot,
        "add_H_cols": add_H_cols,
        "compute_hessians": False,
        "estimate_Sigma": True,
        "return_thetas": False,
        "cross_device_scenario":cross_device_scenario,
        "estimate_sigma_pk_Sigma":True,
        "print_global_parameters":True
    }

    # Run simulations and collect results
    dict_estimations_rct = {}
    for simulation_block in range(n_blocks):
        # RCT
        if specific_parameters_rct is not None:
            params_of_simulation_rct = params_of_simulation.copy()
            estimates_block_rct = Simulations_Fed(
                clients_parameters_rct,
                **params_of_simulation_rct,
            ).estimate_tau(
                **params_of_estimation,
            )
            dict_estimations_rct[simulation_block] = estimates_block_rct

    # Initialize a defaultdict to handle merging
    if specific_parameters_rct is not None:
        merged_estimations_rct = defaultdict(list) 

    def merge_dicts(source, target):
        for key, value in source.items():
            if value is None:  # Skip None values
                continue
            
            if isinstance(value, dict):
                if key not in target or target[key] is None:  # If the target is None, replace it with a dict
                    target[key] = defaultdict(list)
                merge_dicts(value, target[key])
            else:
                if key not in target or target[key] is None:  # If the target is None, replace it with a list
                    target[key] = []
                target[key].extend(value)


    # Merge all simulation block results 
    if specific_parameters_rct is not None:
        for block in dict_estimations_rct.values():
            merge_dicts(block, merged_estimations_rct)


    if specific_parameters_rct is not None:
        merged_estimations_rct = convert_to_dict(merged_estimations_rct)

    if plot:
        if specific_parameters_rct is not None :
            df_tau_hat_rct = Plotter(clients_parameters_rct).plot_estimator(
                dict_estimations=merged_estimations_rct,
                estimator="g_formula",
                scenario_name="RCT",
                hide_legend=False,
                loc_legend="upper right",
                set_dashed_line_to="expectancy_of_tau",
                intercepts_shared_for_legend=True,
                show_locals=show_locals
            )
    return merged_estimations_rct

### Homogeneous

In [ ]:
# Shared parameters
dim_x = 10
mu = [1 for i in range(int(dim_x/2))] + [-1 for i in range(int(dim_x/2))]
sigma = np.eye(dim_x) + 0.5 - np.eye(dim_x) / 2
beta_1 = [2.5 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_0 = [2 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_1[0] = beta_1[0] + 0.15
Y_is_linear = True
sigma2 = 1
n = 150

p_k = 0.5
tau_k_var = 0
random_intercept = False

# Regroup parameters
shared_parameters_dict = {
    "dim_x": dim_x,
    "mean_covariates": mu,
    "cov_covariates": sigma,
    "beta_1": beta_1,
    "beta_0": beta_0,

    "Y_is_linear": Y_is_linear,
    "sample_size": n,
    "sigma2": sigma2,
    "p_k": p_k,
    "tau_k_var": tau_k_var,
    "random_intercept": random_intercept,
}

specific_parameters_rct = {
    "client1": {"sample_size":5*dim_x, "p_k":0.5},
    "client2": {"sample_size":5*dim_x, "p_k":0.5},
    "client3": {"sample_size":5*dim_x, "p_k":0.5},
    "client4": {"sample_size":5*dim_x, "p_k":0.5},
    "client5": {"sample_size":5*dim_x, "p_k":0.5},
}

n_simus = 1000
n_blocks = 1

merged_estimations_rct = run_simulation(shared_parameters_dict, specific_parameters_rct, None, n_simus, n_blocks, show_locals=True, plot=True)


### Imbalanced

In [ ]:
# Shared parameters
dim_x = 10
mu = [1 for i in range(int(dim_x/2))] + [-1 for i in range(int(dim_x/2))]
sigma = np.eye(dim_x) + 0.5 - np.eye(dim_x) / 2
beta_1 = [2.5 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_0 = [2 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_1[0] = beta_1[0] + 0.15
Y_is_linear = True
sigma2 = 1
n = 150

p_k = 0.5
tau_k_var = 0
random_intercept = False

# Regroup parameters
shared_parameters_dict = {
    "dim_x": dim_x,
    "mean_covariates": mu,
    "cov_covariates": sigma,
    "beta_1": beta_1,
    "beta_0": beta_0,

    "Y_is_linear": Y_is_linear,
    "sample_size": n,
    "sigma2": sigma2,
    "p_k": p_k,
    "tau_k_var": tau_k_var,
    "random_intercept": random_intercept,
}

specific_parameters_rct = {
    "client1": {"sample_size":400*dim_x, "p_k":0.5},
    "client2": {"sample_size":25*dim_x, "p_k":0.5},
    "client3": {"sample_size":25*dim_x, "p_k":0.5},
    "client4": {"sample_size":25*dim_x, "p_k":0.5},
    "client5": {"sample_size":25*dim_x, "p_k":0.5},
}

n_simus = 1000
n_blocks = 1

merged_estimations_rct = run_simulation(shared_parameters_dict, specific_parameters_rct, None, n_simus, n_blocks, show_locals=True, plot=True)


### Different Treatment Assignments

In [ ]:
# Shared parameters
dim_x = 10
mu = [1 for i in range(int(dim_x/2))] + [-1 for i in range(int(dim_x/2))]
sigma = np.eye(dim_x) + 0.5 - np.eye(dim_x) / 2
beta_1 = [2.5 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_0 = [2 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_1[0] = beta_1[0] + 0.15
Y_is_linear = True
sigma2 = 1
n = 150

p_k = 0.5
tau_k_var = 0
random_intercept = False

# Regroup parameters
shared_parameters_dict = {
    "dim_x": dim_x,
    "mean_covariates": mu,
    "cov_covariates": sigma,
    "beta_1": beta_1,
    "beta_0": beta_0,

    "Y_is_linear": Y_is_linear,
    "sample_size": n,
    "sigma2": sigma2,
    "p_k": p_k,
    "tau_k_var": tau_k_var,
    "random_intercept": random_intercept,
}

specific_parameters_rct = {
    "client1": {"sample_size":25*dim_x, "p_k":0.3},
    "client2": {"sample_size":25*dim_x, "p_k":0.3},
    "client3": {"sample_size":25*dim_x, "p_k":0.7},
    "client4": {"sample_size":25*dim_x, "p_k":0.7},
    "client5": {"sample_size":25*dim_x, "p_k":0.7},
}

n_simus = 1000
n_blocks = 1

merged_estimations_rct = run_simulation(shared_parameters_dict, specific_parameters_rct, None, n_simus, n_blocks, show_locals=True, plot=True)


### Different Means and Sigmas
Careful: meta IVW is biased

In [ ]:
# Shared parameters
dim_x = 10
mu = [1 for i in range(int(dim_x/2))] + [-1 for i in range(int(dim_x/2))]
sigma = np.eye(dim_x) + 0.5 - np.eye(dim_x) / 2
beta_1 = [2.5 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_0 = [2 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_1[0] = beta_1[0] + 0.15
Y_is_linear = True
sigma2 = 1
n = 150

p_k = 0.5
tau_k_var = 0
random_intercept = False

# Regroup parameters
shared_parameters_dict = {
    "dim_x": dim_x,
    "mean_covariates": mu,
    "cov_covariates": sigma,
    "beta_1": beta_1,
    "beta_0": beta_0,

    "Y_is_linear": Y_is_linear,
    "sample_size": n,
    "sigma2": sigma2,

    "p_k": p_k,
    "tau_k_var": tau_k_var,
    "random_intercept": random_intercept,
}

mu1 = [1 for i in range(int(dim_x/2))] + [-1 for i in range(int(dim_x/2))]
mu2 = [-1 for i in range(int(dim_x/2))] + [1 for i in range(int(dim_x/2))]
mu3 = [0 for i in range(int(dim_x))]
mu4 = [.5 for i in range(int(dim_x/2))] + [-1 for i in range(int(dim_x/2))]
mu5 = [1.2 for i in range(int(dim_x/2))] + [.8 for i in range(int(dim_x/2))]

sigma_second = sigma.copy() * 20 + .5 * np.eye(dim_x) -.5
sigma_third = sigma.copy() * .02 + .7 * np.eye(dim_x)
sigma_forth = sigma.copy() * 1 + .5 * np.eye(dim_x) -.15
sigma_five = sigma.copy() * 1.5 + .5 * np.eye(dim_x) -.15
block_matrix = np.ones(4)
# substract block_matrix to sigma at (2,2)
sigma_second[2:6, 2:6] = sigma_second[2:6, 2:6] - block_matrix * .05
sigma_third[1:5, 1:5] = sigma_third[1:5, 1:5] - block_matrix * .1
sigma_third



# Regroup parameters
shared_parameters_dict = {
    "dim_x": dim_x,
    "mean_covariates": mu,
    "cov_covariates": sigma,
    "beta_1": beta_1,
    "beta_0": beta_0,

    "Y_is_linear": Y_is_linear,
    "sample_size": n,
    "sigma2": sigma2,
    "p_k": p_k,
    "tau_k_var": tau_k_var,
    "random_intercept": random_intercept,
}

specific_parameters_rct = {
    "client1": {"sample_size":25*dim_x, "mean_covariates": mu1, "cov_covariates": sigma, 'p_k':0.3},
    "client2": {"sample_size":25*dim_x, "mean_covariates": mu2, "cov_covariates": sigma_second, 'p_k':0.3},
    "client3": {"sample_size":25*dim_x, "mean_covariates": mu3, "cov_covariates": sigma_third, 'p_k':0.3},
    "client4": {"sample_size":25*dim_x, "mean_covariates": mu4, "cov_covariates": sigma_forth, 'p_k':0.8},
    "client5": {"sample_size":25*dim_x, "mean_covariates": mu5 , "cov_covariates": sigma_five, 'p_k':0.8},
}

n_simus = 1000
n_blocks = 1

merged_estimations_rct = run_simulation(shared_parameters_dict, specific_parameters_rct, None, n_simus, n_blocks, show_locals=True, plot=True,)


### Different intercepts

##### No adjustment

In [ ]:
# Shared parameters
dim_x = 10
mu = [1 for i in range(int(dim_x/2))] + [-1 for i in range(int(dim_x/2))]
sigma = np.eye(dim_x) + 0.5 - np.eye(dim_x) / 2
beta_1 = [2.5 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_0 = [2 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_1[0] = beta_1[0] + 0.15
Y_is_linear = True
sigma2 = 1
n = 150

p_k = 0.5
tau_k_var = 0
random_intercept = False

# Regroup parameters
shared_parameters_dict = {
    "dim_x": dim_x,
    "mean_covariates": mu,
    "cov_covariates": sigma,
    "beta_1": beta_1,
    "beta_0": beta_0,

    "Y_is_linear": Y_is_linear,
    "sample_size": n,
    "sigma2": sigma2,
    "p_k": p_k,
    "tau_k_var": tau_k_var,
    "random_intercept": random_intercept,
}

specific_parameters_rct = {
    "client1": {"sample_size":20*dim_x, "h_k": 30},
    "client2": {"sample_size":20*dim_x, "h_k": -5},
    # "client3": {"sample_size":20*dim_x, "h_k": -1},
    # "client4": {"sample_size":20*dim_x, "h_k": 30},
    # "client5": {"sample_size":20*dim_x, "h_k": 2},
}

n_simus = 1000
n_blocks = 1

merged_estimations_rct = run_simulation(shared_parameters_dict, specific_parameters_rct, None, n_simus, n_blocks, show_locals=True, plot=True)


##### Adjustment

In [ ]:
### Careful ! The number of communications T required grows as the difference between the center effects increases

# Shared parameters
dim_x = 10
tau_is_constant = True
mu = [1 for i in range(int(dim_x/2))] + [-1 for i in range(int(dim_x/2))]
sigma = np.eye(dim_x) + 0.5 - np.eye(dim_x) / 2
beta_1 = [2.5 * i / dim_x - 2 for i in range(0, dim_x + 1)]
beta_0 = [2 * i / dim_x - 2 for i in range(0, dim_x + 1)]

tau = 0
beta_1[0] = beta_1[0] + 0.15
Y_is_linear = True
sigma2 = 1
n = 150

p_k = 0.5
tau_k_var = 0
random_intercept = False

# Regroup parameters
shared_parameters_dict = {
    "dim_x": dim_x,
    "mean_covariates": mu,
    "cov_covariates": sigma,
    "beta_1": beta_1,
    "beta_0": beta_0,

    "Y_is_linear": Y_is_linear,
    "sample_size": n,
    "sigma2": sigma2,

    "p_k": p_k,
    "tau_k_var": tau_k_var,
    "random_intercept": random_intercept,
}

specific_parameters_rct = {
    "client1": {"sample_size":20*dim_x, "h_k": 30},
    "client2": {"sample_size":20*dim_x, "h_k": -5},
    # "client3": {"sample_size":20*dim_x, "h_k": -1},
    # "client4": {"sample_size":20*dim_x, "h_k": 3},
    # "client5": {"sample_size":20*dim_x, "h_k": 2},
}

n_simus = 1000
n_blocks = 1

merged_estimations_rct = run_simulation(shared_parameters_dict, specific_parameters_rct, None, n_simus, n_blocks, meta_IVW=True, intercepts_shared_for_shot=False, add_H_cols=True,)
